# Point - to - point relations

In [ ]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

Functions for visualisation:

In [ ]:
def regular_on_rectangle(grid, random_component, x_lim, y_lim):
    dx = (x_lim[1] - x_lim[0])/grid[0]
    dy = (y_lim[1] - y_lim[0])/grid[1]
    mx = np.linspace(x_lim[0] + dx/2, x_lim[1] - dx/2, grid[0])
    my = np.linspace(y_lim[0] + dy/2, y_lim[1] - dy/2, grid[1])
    px,py=np.meshgrid(mx,my)
    rx = np.random.rand(grid[1],grid[0])* dx - dx/2
    ry = np.random.rand(grid[1],grid[0])* dy - dy/2
    px=px+random_component*rx
    py=py+random_component*ry
    p={'X':px.flatten(), 'Y':py.flatten()}
    points=pd.DataFrame(p)
    return points
def homogeneous_poisson_on_rectangle(intensity, x_lim, y_lim):
    a = (abs(x_lim[1]-x_lim[0]))*(abs(y_lim[1]-y_lim[0]))
    ex = intensity*a
    n = np.random.poisson(lam=ex, size=1)
    nn=n[0]
    x=np.random.rand(nn)*(x_lim[1]-x_lim[0])+x_lim[0]
    y=np.random.rand(nn)*(y_lim[1]-y_lim[0])+y_lim[0]
    p={"X":x.flatten(),"Y":y.flatten()}
    points=pd.DataFrame(p)
    return points
def materna_on_rectangle(parent_intensity, daughter_intensity, cluster_radius, x_lim, y_lim):
    x_lim2 = [0,0]
    y_lim2 = [0,0]
    x_lim2[0] = x_lim[0] - cluster_radius;
    x_lim2[1] = x_lim[1] + cluster_radius;
    y_lim2[0] = y_lim[0] - cluster_radius;
    y_lim2[1] = y_lim[1] + cluster_radius;
    xWym = x_lim2[1] - x_lim2[0];
    yWym = y_lim2[1] - y_lim2[0];
    a2 = xWym * yWym;
    Points = np.random.poisson (a2 * parent_intensity);
    x_P = x_lim2[0] + xWym * np.random.uniform(0, 1, Points);
    y_P = y_lim2[0] + yWym * np.random.uniform(0, 1, Points);
    
    PointsCluster = np.random.poisson(daughter_intensity, Points)
    PointsClusterSum=sum(PointsCluster)
    
    theta = 2 * np. pi* np.random.uniform(0, 1, PointsClusterSum);
    ro = cluster_radius* np.sqrt(np.random.uniform(0, 1, PointsClusterSum));
    
    x_KO = ro*np.cos(theta);
    y_KO = ro*np.sin(theta);
    
    x_P2=np.repeat(x_P, PointsCluster);
    y_P2=np.repeat(y_P, PointsCluster);
    
    x_P2 = x_P2 + x_KO;
    y_P2 = y_P2 + y_KO;

    cut = ((x_P2 >= x_lim[0]) & (x_P2 <= x_lim[1]) & (y_P2 >= y_lim[0]) & (y_P2 <= y_lim[1]));
    x_P2 = x_P2[cut];
    y_P2 = y_P2[cut];
    p4=pd.DataFrame({'X':x_P2, 'Y':y_P2})
    return p4

In [ ]:
points_HP=homogeneous_poisson_on_rectangle(2.5, [0,10], [0,10])
points_R=regular_on_rectangle([15,15], 0.75, [0,10],[0,10])
points_M=materna_on_rectangle(0.3, 5, 0.75, [0,10], [0,10])
points_test=homogeneous_poisson_on_rectangle(20, [0,10], [0,10])

## Morishita plot

In [ ]:
def morishita_plot_data(points, bins_list, x_lim, y_lim):
    BD=[]
    MI=[]
    for i in bins_list:
        H,y_lims,x_lims = np.histogram2d(points["Y"], points["X"], bins=[int(i[0]),int(i[1])],range=[[x_lim[0], x_lim[1]], [y_lim[0], y_lim[1]]])
        r=[y_lims[1]-y_lims[0]]
        BD.extend(r)
        s=[i[0]*i[1]*np.sum((H*(H-1))/(points["X"].size*(points["X"].size-1)))]
        MI.extend(s)
    df=pd.DataFrame({'BD':BD,'MI':MI})
    return df

### Generating exeplary process

In [ ]:
bins_list=np.linspace([2,2],[40,40],39)

m_HP=morishita_plot_data(points_HP, bins_list, [0,10], [0,10])
m_R=morishita_plot_data(points_R, bins_list, [0,10], [0,10])
m_M=morishita_plot_data(points_M, bins_list, [0,10], [0,10])

### Visualisation

In [ ]:
fig, ax=plt.subplots(2,3,figsize=(15,10))
fig.suptitle("Morishita plots with original point processes", fontsize=25)
ax[0,0].set_title("Homogenous Poisson")
sns.scatterplot(data=points_HP,x="X",y="Y",ax=ax[0,0])
ax[0,0].set_aspect('equal', 'box')

ax[0,1].set_title("Regular Poisson")
sns.scatterplot(data=points_R,x="X",y="Y",ax=ax[0,1])
ax[0,1].set_aspect('equal', 'box')

ax[0,2].set_title("Materna process")
sns.scatterplot(data=points_M,x="X",y="Y",ax=ax[0,2])
ax[0,2].set_aspect('equal', 'box')

ax[1,0].set_title("Morishita plot - Homogenous Poisson")
s=sns.scatterplot(data=m_HP,x="BD",y="MI",ax=ax[1,0])
s.set_ylim(-1,3)
ax[1,0].set_aspect('equal', 'box')

ax[1,1].set_title("Morishita plots - Regular Poisson")
s=sns.scatterplot(data=m_R,x="BD",y="MI",ax=ax[1,1])
s.set_ylim(-1,3)
ax[1,1].set_aspect('equal', 'box')

ax[1,2].set_title("Morishita plots - Materna Process")
s=sns.scatterplot(data=m_NH,x="BD",y="MI",ax=ax[1,2])
s.set_ylim(-1,3)
ax[1,2].set_aspect('equal', 'box')

## Fry plot

In [ ]:
def fry_plot_data(points):
    x=[]
    y=[]
    for i in range(0,np.size(points["X"])):
        pom=points["X"].drop(i)
        x.extend(pom-points["X"][i])
        pom=points["Y"].drop(i)
        y.extend(pom-points["Y"][i])
    return pd.DataFrame({"X":x, "Y":y})

### Generating exeplary process

In [ ]:
fry_HP=fry_plot_data(points_HP)
fry_R=fry_plot_data(points_R)
fry_M=fry_plot_data(points_M)

### Visualisation

In [ ]:
fig, ax=plt.subplots(2,3,figsize=(15,12))
fig.suptitle("Fry plots with original point processes", fontsize=25)
ax[0,0].set_title("Homogenous Poisson")
sns.scatterplot(data=points_HP,x="X",y="Y",ax=ax[0,0])

ax[0,1].set_title("Regular Poisson")
sns.scatterplot(data=points_R,x="X",y="Y",ax=ax[0,1])

ax[0,2].set_title("Materna process")
sns.scatterplot(data=points_M,x="X",y="Y",ax=ax[0,2])

ax[1,0].set_title("Fry plot - Homogenous Poisson")
sns.scatterplot(data=fry_HP,x="X",y="Y",ax=ax[1,0], alpha=0.05)

ax[1,1].set_title("Fry plot - Regular Poisson")
sns.scatterplot(data=fry_R,x="X",y="Y",ax=ax[1,1],alpha=0.01)

ax[1,2].set_title("Fry plot - Materna process")
sns.scatterplot(data=fry_M,x="X",y="Y",ax=ax[1,2],alpha=0.15,s=10)

## G function

In [ ]:
def g_function(points):
    m=[]
    g=[]
    for i in range(points['X'].size):
        px0=points['X'][i]-points['X'].drop(i)
        py0=points['Y'][i]-points['Y'].drop(i)
        odl_pom=(px0**2+py0**2)**0.5
        m.append(odl_pom.min())
    m=np.sort(m)
    d=np.unique(m)
    ss=m.size
    for i in d:
        s=np.sum(m<=i)
        g.append(s/ss)
    d=np.insert(d,0,0.0)
    g.insert(0,0)
    return pd.DataFrame({"D":d, "G":g})
    
def g_function_poisson(d, intensity):
    g=1-np.exp(-intensity*np.pi*d*d)
    return pd.DataFrame({"D":d, "G":g})

### Generating exeplary process

In [ ]:
p_HP_g=g_function(points_HP)
p_R_g=g_function(points_R)
p_M_g=g_function(points_M)

p_HP_gp=g_function_poisson(p_HP_g["D"],2.5)
p_R_gp=g_function_poisson(p_R_g["D"],2.5)
p_M_gp=g_function_poisson(p_M_g["D"],2.5)

### Visualisation

In [ ]:
fig, ax=plt.subplots(2,3,figsize=(15,15))
fig.suptitle("G function", fontsize=25)

ax[0,0].set_title("Homogenous Poisson")
sns.scatterplot(data=points_HP,x="X",y="Y",ax=ax[0,0])

ax[0,1].set_title("Regular Poisson")
sns.scatterplot(data=points_R,x="X",y="Y",ax=ax[0,1])

ax[0,2].set_title("Materna process")
sns.scatterplot(data=points_M,x="X",y="Y",ax=ax[0,2])


ax[1,0].set_title("G function - Homogenous Poisson")
sns.lineplot(data=p_HP_gp,x="D",y="G",ax=ax[1,0])
sns.lineplot(data=p_HP_g,x="D",y="G",ax=ax[1,0])
ax[1,0].legend(["theoretical G function","G function for evaluated points"])

ax[1,1].set_title("G function - Regular Poisson")
sns.lineplot(data=p_R_gp,x="D",y="G",ax=ax[1,1])
sns.lineplot(data=p_R_g,x="D",y="G",ax=ax[1,1])
ax[1,1].legend(["theoretical G function","G function for evaluated points"])

ax[1,2].set_title("G function - Materna process")
sns.lineplot(data=p_M_gp,x="D",y="G",ax=ax[1,2])
sns.lineplot(data=p_M_g,x="D",y="G",ax=ax[1,2])
ax[1,2].legend(["theoretical G function","G function for evaluated points"])

## F function

In [ ]:
def f_function(points, test_points):
    m = []
    f = []
    for i in range(test_points['X'].size):
        x_t = test_points["X"][i] - points["X"]
        y_t = test_points["Y"][i] - points["Y"]
        euk = (x_t**2 + y_t**2)**0.5
        m.append(euk.min())
    m=np.sort(m)
    d=np.unique(m)
    ss=m.size
    for i in d:
        s=np.sum(m<=i)
        f.append(s/ss)
    d=np.insert(d,0,0.0)
    f.insert(0,0)
    return pd.DataFrame({"D":d, "F":f})
    
def f_function_poisson(d, intensity):
    f=1-np.exp(-intensity*np.pi*d*d)
    return pd.DataFrame({"D":d, "F":f})

### Generating exeplary process

In [ ]:
p_HP_f=f_function(points_HP,points_test)
p_R_f=f_function(points_R,points_test)
p_M_f=f_function(points_M,points_test)

p_HP_fp=f_function_poisson(p_HP_f["D"],2.5)
p_R_fp=f_function_poisson(p_R_f["D"],2.5)
p_M_fp=f_function_poisson(p_M_f["D"],2.5)

### Visualisation

In [ ]:
fig, ax=plt.subplots(2,3,figsize=(15,12))
fig.suptitle("F function", fontsize=25)
ax[0,0].set_title("Homogenous Poisson")
sns.scatterplot(data=points_HP,x="X",y="Y",ax=ax[0,0])


ax[0,1].set_title("Regular Poisson")
sns.scatterplot(data=points_R,x="X",y="Y",ax=ax[0,1])

ax[0,2].set_title("Materna process")
sns.scatterplot(data=points_M,x="X",y="Y",ax=ax[0,2])

ax[1,0].set_title("F function - Homogenous Poisson")
sns.lineplot(data=p_HP_fp,x="D",y="F",ax=ax[1,0])
sns.lineplot(data=p_HP_f,x="D",y="F",ax=ax[1,0])
ax[1,0].legend(["theoretical F function","F function for evaluated points"])

ax[1,1].set_title("F function - Regular Poisson")
sns.lineplot(data=p_R_fp,x="D",y="F",ax=ax[1,1])
sns.lineplot(data=p_R_f,x="D",y="F",ax=ax[1,1])
ax[1,1].legend(["theoretical F function","F function for evaluated points"])

ax[1,2].set_title("F function - Materna process")
sns.lineplot(data=p_M_fp,x="D",y="F",ax=ax[1,2])
sns.lineplot(data=p_M_f,x="D",y="F",ax=ax[1,2])
ax[1,2].legend(["theoretical F function","F function for evaluated points"])